# Searching new ontology concepts in DBPedia 

In [ ]:
!pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON

# Specify the DBPedia endpoint
sparql = SPARQLWrapper("http://pt.dbpedia.org/sparql")

     |████████████████████████████████| 231 kB 12.5 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


## Searching by /resource/Category:

### Hyponyms

skos:broader é a propriedade que delimita o que é mais amplo. É possível botar um "+" ou {1,n} tamanho de vezes para *recursão* * disclamer: Por algum motivo, a query interpreta o skos:broader como as subclasses, ao invés das superclasses *

In [ ]:
# searching 
sparql.setQuery("""
    SELECT ?hyponyms
    WHERE {
        ?hyponyms skos:broader <http://pt.dbpedia.org/resource/Categoria:Hotéis> .

    }   
""")

# Convert results to JSON format
sparql.setReturnFormat(JSON)
hypo = sparql.query().convert()
hypo

{'head': {'link': [], 'vars': ['hyponyms']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'hyponyms': {'type': 'uri',
     'value': 'http://pt.dbpedia.org/resource/Categoria:Hotéis_por_cidade'}},
   {'hyponyms': {'type': 'uri',
     'value': 'http://pt.dbpedia.org/resource/Categoria:Hotéis_fictícios'}},
   {'hyponyms': {'type': 'uri',
     'value': 'http://pt.dbpedia.org/resource/Categoria:Hotéis_por_país'}},
   {'hyponyms': {'type': 'uri',
     'value': 'http://pt.dbpedia.org/resource/Categoria:Redes_de_hotéis'}},
   {'hyponyms': {'type': 'uri',
     'value': 'http://pt.dbpedia.org/resource/Categoria:Tipos_de_hotéis'}},
   {'hyponyms': {'type': 'uri',
     'value': 'http://pt.dbpedia.org/resource/Categoria:Hoteleiros'}},
   {'hyponyms': {'type': 'uri',
     'value': 'http://pt.dbpedia.org/resource/Categoria:Hotéis_extintos'}}]}}

### Hypernyms

Usando novamente o skos:broader, porém com o sentido de pegar as superclasses, ou categorias que estão acima na hierarquia. 

O operador "^" faz a função de 'subir' um nivel a cima da árvore.

In [ ]:
sparql.setQuery("""
    SELECT ?hyponyms
    WHERE {
        ?hyponyms ^skos:broader <http://pt.dbpedia.org/resource/Categoria:Hotéis> .



    } 
""")

# Convert results to JSON format
sparql.setReturnFormat(JSON)
hyper = sparql.query().convert()
hyper

{'head': {'link': [], 'vars': ['hyponyms']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'hyponyms': {'type': 'uri',
     'value': 'http://pt.dbpedia.org/resource/Categoria:Construções'}},
   {'hyponyms': {'type': 'uri',
     'value': 'http://pt.dbpedia.org/resource/Categoria:Meios_de_hospedagem'}}]}}

## Searching by /resource

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# dataframe creation


labels =['wikiPageLink', 'subject', 'isPrimaryTopicOf', 'wikiPageRedirects', 'is_wikiPageLink_of']

modify_wikiLink=  pd.DataFrame(columns = [labels[0], 'concept']) #done
modify_subject=  pd.DataFrame(columns = [labels[1], 'concept']) #done
modify_isPrimaryTopicOf =  pd.DataFrame(columns = [labels[2], 'concept']) #failed
modify_wikiPageRedirects =  pd.DataFrame(columns = [labels[3], 'concept']) #done
modify_is_wikiPageLink_of =  pd.DataFrame(columns = [labels[4], 'concept']) #done





In [ ]:
# reading the ontology's concepts file

with open('/work/concepts_hoteis.txt', "r") as file1:
    conceptsList = file1.readlines()

for x in range(len(conceptsList)):
    conceptsList[x] = (conceptsList[x].rstrip("\n"))



In [ ]:
for concept in conceptsList:
    
    conceptWord = (concept).capitalize()

    queryWord = "_".join(conceptWord.split(" "))

    sparql.setQuery("""
        PREFIX dcterms: <http://purl.org/dc/terms/>
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>
        SELECT ?is_wikiPageLink_of
        WHERE {
            ?is_wikiPageLink_of ^dbo:wikiPageRedirects <http://pt.dbpedia.org/resource/""" + conceptWord + """> .



        } 
    """)

    # Convert results to JSON format
    sparql.setReturnFormat(JSON)

    try:
        dbpedia = sparql.query().convert()

        ptopic = dbpedia['results']['bindings']




        #spliting to get the term 
        for data in ptopic:
            #print(data)
            url = data['is_wikiPageLink_of']['value']
            concept_url = url.split("http://pt.dbpedia.org/resource/")[1]
            #print(concept_url)
            series_obj = pd.Series( [concept_url, concept], index=modify_is_wikiPageLink_of.columns )
            # Add a series as a row to the dataframe  
            modify_is_wikiPageLink_of = modify_is_wikiPageLink_of.append( series_obj, ignore_index=True)
            #print("conceito:" + concept + "; tem dbpedia? sim")


    except Exception as e:
        conceptsList.remove(concept)
        #print("conceito:" + concept + "; tem dbpedia? não")


In [ ]:
print("Tamanho reduzido para : ", len(conceptsList) )
conceptsList

Tamanho reduzido para :  160


['aparência',
 'atendimento',
 'conforto',
 'classe superior',
 'hotéis luxo',
 'hotéis padrão',
 'primeira classe',
 'turista',
 'uma estrela',
 'três estrelas',
 'cinco estrelas',
 'custo-benefício',
 'albergue',
 'apartamento',
 'cama e café da manhã',
 'hotel',
 'hotel aquático',
 'hotel casa da árvore',
 'hotel cápsula',
 'hotel fazenda',
 'hotel histórico',
 'motel',
 'pousada',
 'pensão',
 'funcionários',
 'babá',
 'funcionário da lavanderia',
 'funcionário da piscina',
 'funcionário da área de lazer',
 'garçom',
 'gerente',
 'manobrista',
 'mensageiro',
 'motorista',
 'porteiro',
 'recreacionista',
 'horário',
 'horário do estacionamento',
 'horário do spa',
 'acesso para deficientes',
 'aquecedor de toalhas',
 'balança',
 'chuveiro',
 'pia',
 'roupão',
 'torneira',
 'hidromassagem',
 'ar-condicionado',
 'cama',
 'chaleira',
 'cortina',
 'ferro de passar roupa',
 'iluminação',
 'isolamento acústico',
 'móveis',
 'guarda-roupa',
 'televisão',
 'tomada',
 'tábua de passar roupa',

In [ ]:
modify_wikiPageRedirects.head()

,wikiPageRedirects,concept


In [ ]:
modify_is_wikiPageLink_of.to_csv (r'./export_dataframe.csv', index = False, header=True)


In [ ]:
len(modify_is_wikiPageLink_of)

4

###  Dataframes Sizes 

In [ ]:
isWikiPageLen = len(pd.read_csv('/work/is_wikiPageLink_of_dataframe.csv'))  
subjectsLen = len(pd.read_csv('/work/subjects_dataframe.csv'))  
wikiPageLinkLen = len(pd.read_csv('/work/wikipagelink_dataframe.csv'))  
wikiPageRedirectsLen = len(pd.read_csv('/work/wikiPageRedirects_dataframe.csv'))  

print("isWikiPageLink's length: ", isWikiPageLen)
print("subjects's length: ", subjectsLen)
print("WikiPageLinks's length: ", wikiPageLinkLen)
print("wikiPageRedirects's length: ", wikiPageRedirectsLen)

print("----------------------------------------------")

print("TOTAL LENGTH: ", isWikiPageLen+subjectsLen+ wikiPageLinkLen+wikiPageRedirectsLen)


isWikiPageLink's length:  7
subjects's length:  19
WikiPageLinks's length:  374
wikiPageRedirects's length:  15
----------------------------------------------
TOTAL LENGTH:  415


### Reading the TripAdivisor dictionary

In [ ]:
# reading the dictionary file

with open('/work/dict.txt', "r") as file1:
    dic = file1.readlines()

for x in dic:
    tripAdvisor = eval(x)

print("total len:", len(tripAdvisor))

noun = 0
verb = 0
aux = 0
outro = 0
for key in tripAdvisor.values():
    if(key == 'NOUN'):
        noun+=1
    elif(key == 'VERB'):
        verb+=1
    elif(key == 'AUX'):
        aux+=1
    else:
        outro+=1

print("number of verbs: ", verb)
print("number of nouns: ", noun)
print("number of auxs: ", aux)
print("number of outros: ", outro)

total len: 11356
number of verbs:  4052
number of nouns:  7109
number of auxs:  195
number of outros:  0


## Doc2vec experiments

### Extracting articles from Wikipedia 

In [ ]:
!pip install bs4

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import bs4
import sys
import requests
import posixpath

In [ ]:
for concept in conceptsList:
    conceptWord = (concept).capitalize()
    queryWord = "_".join(conceptWord.split(" "))
    wiki_page = queryWord
    res = requests.get(f'https://pt.wikipedia.org/wiki/{wiki_page}' )
    
    try:
        res.raise_for_status()
        wiki = bs4.BeautifulSoup(res.text,"html.parser")
        
        completeName = posixpath.join('/work/articles', wiki_page+".txt")

        # open a file named as your wiki page in write mode
        with open(completeName, "w", encoding="utf-8") as f:
            for i in wiki.select('p'):
                # write each paragraph to the file
                f.write(i.getText())
    
    except Exception as e:
        #print(e)

In [ ]:
## MERGIN THE ARTICLES:
from os import walk

_, _, filenames = next(walk('/work/articles'))

with open('/work/output_articles/merge.txt', 'w') as outfile:
    for fname in filenames:
        with open('/work/articles/'+fname) as infile:
            for line in infile:
                outfile.write(line)

## Creating the dataset (first attempt)

Reviwing dbpedia dataframes:

In [ ]:
#is_wikipagelink_of - done - type of relation: none 
#subject            - done - type of relation: hypernyms
#wikipagelink       - done - type of relation: none 
#redirects          - done - type of relation: none

In [ ]:
# dataframes w/ new sizes:

isWikiPage = (pd.read_csv('/work/dbpedia_dataframes/is_wikiPageLink_of_dataframe.csv'))  
subject = (pd.read_csv('/work/dbpedia_dataframes/subjects_dataframe.csv'))  
wikiPageLink = (pd.read_csv('/work/dbpedia_dataframes/wikipagelink_dataframe.csv'))  
wikiPageRedirects = (pd.read_csv('/work/dbpedia_dataframes/wikiPageRedirects_dataframe.csv'))


In [ ]:
# add the relations
isWikiPage['label'] = 'none'
subject['label'] = 'hypernym'
wikiPageLink['label'] = 'none'
wikiPageRedirects['label'] = 'none'

In [ ]:
# renaming the columns 
isWikiPage = isWikiPage.rename(columns={'is_wikiPageLink_of':'dbpedia_term'})
subject = subject.rename(columns={'subject':'dbpedia_term'})
wikiPageLink = wikiPageLink.rename(columns={'wikiPageLink':'dbpedia_term'})
wikiPageRedirects = wikiPageRedirects.rename(columns={'wikiPageRedirects':'dbpedia_term'})


In [ ]:
dbpediaDataset= pd.concat([isWikiPage, subject, wikiPageLink, wikiPageRedirects])
dbpediaDataset.head()

,dbpedia_term,concept,label
0,Turismo,turista,none
1,Relação_de_custo-benefício,custo-benefício,none
2,Trabalhador,funcionários,none
3,Valet,manobrista,none
4,Pia,lavatório,none


In [ ]:
dbpediaDataset.to_csv (r'./dbpedia_dataframes/dbpedia_dataset.csv', index = False, header=True)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a5cf3ea2-750b-4fc7-8242-5eefd4212f21' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>